# Create a weather dataframe that captures external factors

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import openmeteo_requests
import requests_cache
from retry_requests import retry


In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [63.43, 62.47, 62.74, 64.47, 62.57],
	"longitude": [10.39, 6.15, 7.16, 11.49, 11.38],
	"start_date": "2024-01-01",
	"end_date": "2024-12-31",
	"hourly": "temperature_2m",
	"timezone": "GMT"
}
responses = openmeteo.weather_api(url, params=params)
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

							# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


'\nresponse = responses[0]\nprint(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")\nprint(f"Elevation {response.Elevation()} m asl")\nprint(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")\nprint(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")\n\n\t\t\t\t\t\t\t# Process hourly data. The order of variables needs to be the same as requested.\nhourly = response.Hourly()\nhourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()\n\nhourly_data = {"date": pd.date_range(\n\tstart = pd.to_datetime(hourly.Time(), unit = "s", utc = True),\n\tend = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),\n\tfreq = pd.Timedelta(seconds = hourly.Interval()),\n\tinclusive = "left"\n)}\n\nhourly_data["temperature_2m"] = hourly_temperature_2m\n\nhourly_dataframe = pd.DataFrame(data = hourly_data)\nprint(hourly_dataframe)\n'

In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Define location names
location_names = {
    (63.43, 10.39): "Trondheim", # Largest city in central Norway
    (62.47, 6.15): "Alesund", # Major coastal city in the western part of NO3
    (62.74, 7.16): "Molde", # Another significant city in the region
    (64.47, 11.49): "Namsos", # Near the northern border of NO3
    (62.57, 11.38): "Røros" # Eastern part of N03, different climate characteristics
}

# Make sure all required weather variables are listed here
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": [63.43, 62.47, 62.74, 64.47, 62.57],
    "longitude": [10.39, 6.15, 7.16, 11.49, 11.38],
    "start_date": "2024-01-01",
    "end_date": "2024-12-31",
    "hourly": "temperature_2m",
    "timezone": "GMT"
}
responses = openmeteo.weather_api(url, params=params)

# Create a base dataframe with the date column using the first response
hourly = responses[0].Hourly()
hourly_data = {"date": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
)}

# Create the main dataframe with the date column
combined_df = pd.DataFrame(data = hourly_data)

# Process each location and add its temperature as a new column
for i, response in enumerate(responses):
    # Get location information
    lat = response.Latitude()
    lon = response.Longitude()
    
    # Use the location name from our mapping
    location_key = (lat, lon)
    print(location_key)
    #location_name = location_names.get(location_key, f"temp_{lat:.2f}N_{lon:.2f}E")
    location_name = location_names[location_key]
    
    # Get temperature data
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    
    # Add to the combined dataframe
    combined_df[location_name] = hourly_temperature_2m

None


KeyError: None

In [24]:
combined_df[combined_df['date'] == '2024-08-07 00:00:00+00:00']

,date,temp_63.44N_10.37E,temp_62.46N_6.36E,temp_62.74N_7.12E,temp_64.50N_11.53E,temp_62.60N_11.65E
5256,2024-08-07 00:00:00+00:00,20.2185,16.3265,16.351,18.835499,13.53


In [25]:
# Define location names
location_names = {
    (63.43, 10.39): "Trondheim", # Largest city in central Norway
    (62.47, 6.15): "Alesund", # Major coastal city in the western part of NO3
    (62.74, 7.16): "Molde", # Another significant city in the region
    (64.47, 11.49): "Namsos", # Near the northern border of NO3
    (62.57, 11.38): "Røros" # Eastern part of N03, different climate characteristics
}

In [30]:
city = location_names[(63.43, 10.39)]

In [31]:
city

'Trondheim'